### Data Collection





In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
import os
import json
from requests_oauthlib import OAuth1Session
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#from newsapi import NewsApiClient
from pathlib import Path
import tweepy as tw
%matplotlib inline
load_dotenv()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jakek\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
df_1 = pd.read_csv('tweets1.csv', usecols = ['id', 'created_at', 'text'])
df_2 = pd.read_csv('tweets2.csv', usecols = ['id', 'created_at', 'text'])
df_3 = pd.read_csv('tweets3.csv', usecols = ['id', 'created_at', 'text'])
df_4 = pd.read_csv('tweets4.csv', usecols = ['id', 'created_at', 'text'])

In [5]:
df_total = pd.concat([df_1,df_2,df_3,df_4])

In [6]:
df_scrub = df_total[df_total['text'].str.contains("$", regex=False)]
df_scrub

,id,created_at,text
31,1.500000e+18,2022-03-12T16:57:32.000Z,Cohen??s Point72 to Redeem $750 Million From ...
74,1.500000e+18,2022-03-11T18:17:03.000Z,*83 COUNTERPARTIES TAKE $1.558 TLN AT FED REVE...
75,1.500000e+18,2022-03-11T18:15:27.000Z,*SEC AWARDS $14M TO WHISTLEBLOWER WHO EXPOSED ...
3,1.500000e+18,2022-03-12T03:04:33.000Z,"@RedRockTrader1 Yes, I missed the short after ..."
5,1.500000e+18,2022-03-12T00:33:56.000Z,Sometimes you just have to ask for the VWAP sq...
...,...,...,...
78,1.500000e+18,2022-03-09T20:25:04.000Z,$CCJ Notable Options Flow in Open Interest (So...
79,1.500000e+18,2022-03-09T20:19:43.000Z,Getting it! $CCJ https://t.co/NiqQcA0bAW
83,1.500000e+18,2022-03-09T18:30:30.000Z,"$SPY\r\n\r\nWhy 414/415 was so important, last..."
87,1.500000e+18,2022-03-09T17:14:08.000Z,"$ABBV highs, tight bull flag forming for what ..."


In [7]:
tw_sentiments = []

for text in df_scrub["text"]:
    try:

        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tw_sentiments.append({
            "text": text,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })

    except AttributeError:
        pass

In [8]:
tw_df = pd.DataFrame(tw_sentiments)

In [9]:
cols = ["text", "compound", "positive", "negative", "neutral"]
tw_df = tw_df[cols]
tw_df = tw_df.sort_values('compound', ascending = False)
tw_df

,text,compound,positive,negative,neutral
64,$ISPO I liked it last time when it went straig...,0.8760,0.459,0.000,0.541
17,$HYMC .50/share ago and 280 million shares tra...,0.8426,0.215,0.000,0.785
99,$CMG notable early strength above its 8-MA\r\n...,0.8126,0.261,0.000,0.739
36,$GFAI just so you know SSR is on -- so they ha...,0.7506,0.144,0.039,0.817
20,Days like today it's nice to have @DilutionTra...,0.7399,0.343,0.000,0.657
...,...,...,...,...,...
100,$NVDA weak down 3.5% under yesterday low after...,-0.6124,0.000,0.294,0.706
67,One of those days you're happy $AGRI borrow wa...,-0.6249,0.152,0.221,0.626
98,$OLLI large Apr. put spread bought\r\n\r\nLate...,-0.8055,0.000,0.248,0.752
30,$GFAI speaking of the devil ????\r\n\r\nNice w...,-0.8205,0.084,0.312,0.603


In [ ]:
# Top 5 tickers with highest compound sentiment analysis 
## ISPO, HYMC, CMG, GFAI and DATS